# 0. Checking CUDA Availability + Making Sure GPU is Utilized

In [3]:
import torch
import math
import numpy
import matplotlib.pyplot as plt

print("PyTorch Version: " + torch.__version__)
print("CUDA Availability: " + str(torch.cuda.is_available()))
print("Device Count: " + str(torch.cuda.device_count()) + "\n")

current = torch.cuda.current_device()

print("Current Device Index: " + str(current))
print("Current Device Object: " + str(torch.cuda.device(current)))
print("Current Device Name: " + torch.cuda.get_device_name(current))

PyTorch Version: 2.4.1+cu124
CUDA Availability: True
Device Count: 1

Current Device Index: 0
Current Device Object: <torch.cuda.device object at 0x7af95486fc40>
Current Device Name: NVIDIA GeForce RTX 4060 Laptop GPU


# 1-1a. Simulate a Function

The function I will be simulating is $ f(x) = sgn(\frac{sin(4x)}{1.1 + cos(2x)}) $.

In [4]:
# List to contain generated function data
function_data = []

# Function representation of the simulated function
def groundtruth_function(x):
    fx = math.sin(4 * x) / (1.1 + math.cos(2 * x))

    if fx == 0:
        return 0
    elif fx > 0:
        return 1
    else:
        return -1

#Get [x, f(x)] pairs for all points between start and end, at the interval indicated by jump_rate
start = -2000
end = 2000
jump_rate = 0.2

while start < end:
    function_data.append([start, groundtruth_function(start)])
    start += jump_rate

#Convert list into PyTorch tensor
function_data = torch.tensor(function_data)

print(function_data.size())

torch.Size([20000, 2])


# 1-1b. Train on Actual Task

# 1-2. Optimization

# 1-3. Generalization